# Numerical explicit method code
This code can price options/derivatives using the numerical explicit method. It can take any path-independent payoff, combined with lower and upper boundaries (but no other path-dependent conditions, although it could easily be modified to do so).<br><br>

### Index
<h4 style="margin-bottom:5px">(1) Most generalized program</h4>
&nbsp;&nbsp;&nbsp;&nbsp; This program performs the explicit method for the widest range of situations and options. Also contains a bilinear interpolation function, to find values for S and tau not contained in the solution grid.

<h4 style="margin-bottom:5px">(2) Efficiently applied to Black-Scholes and other assumptions</h4>
&nbsp;&nbsp;&nbsp;&nbsp; This program has been slightly modified from the generalized one to exclude code and steps that become unnecessary given some extra assumptions.

## (1) Most generalized program
This code can work with any axis (asset and time) scaling, time-dependent or asset-dependent volatility and risk-free rate... and just any generalization I could think of (indicated in the code if necessary).

In [ ]:
import numpy as np
import math

'''
### (3 hashtags) means that this line can be customized for the particular case, and the code is generalized enough to adapt.
'''

# PARAMETERS
# Method parameters
N_S = 100 ### Number of asset steps apart from the 0th
N_tau = 600 ### Number of time steps apart from the 0th
min_topS = 270 ### Minimum value of S high enough to be the upper bound, i.e. V(S) ≈ lim S->inf V(S)

# Financial parameters 1 (customize as needed)
S_0 = 100 ### Current asset value
T = 1 ### In years

# Axis
k = max(min_topS / S_0, (1 + 1 / (N_S // 2)) ** math.ceil(N_S / 2)) ###
S_array = np.array([i * (S_0 / (N_S // 2)) for i in range(N_S // 2)] + [S_0 * k ** (i / math.ceil(N_S / 2)) for i in range(math.ceil(N_S / 2) + 1)]) ###
tau_array = np.array([(T / N_tau) * i for i in range(N_tau + 1)]) ### Time to maturity (in years), so tau_array[0] = 0 is maturity, and tau_array[N_tau] = T is current time

dS = np.array([S_array[i + 1] - S_array[i] for i in range(N_S)])
dtau = np.array([tau_array[i + 1] - tau_array[i] for i in range(N_tau)])

# Financial parameters 2
sigma = np.array([[0.2 for j in range(N_tau + 1)] for i in range(N_S + 1)]) ### Asset- and time-dependent, in years^(-1/2)
r = np.array([0.04 for i in range(N_tau + 1)]) ### Time-dependent, in years^(-1)

# Option parameters (Now set for a vanilla call with strike 100)
def Payoff(S): ###
    return max(S - 100, 0)
def LBoundary(i, j): ###
    return 0
def UBoundary(i, j): ###
    return S_array[i] - 100 * math.exp(-r[j] * tau_array[j])

# Equation parameters (Now set for Black-Scholes PDE)
'''
The method works for any linear homogeneous PDE of the form "Theta + a(S, t)Gamma + b(S, t)Delta + c(S, t)V = 0". The following, a, b, c, are those coefficients/functions.
'''
a = np.array([[(1/2) * (sigma[i, j] * S_array[i]) ** 2 for j in range(N_tau)] for i in range(N_S + 1)]) ###
b = np.array([[r[j] * S_array[i] for j in range(N_tau)] for i in range(N_S + 1)]) ###
c = np.array([[-r[j] for j in range(N_tau)] for i in range(N_S + 1)]) ###

# Convergence constraints
for i in range(N_S):
    dif = min([2 * a[i, j] / abs(b[i, j]) for j in range(N_tau)]) - dS[i]
    if dif < 0:
        print(f'dS, no converge chicos, dS[{i}], {dif}, {dS[i]}') # If it prints, increase N_S

for i in range(N_tau):
    dif = min([dS[j] / (2 * a[j, i]) for j in range(N_S)]) - dtau[i]
    if dif < 0:
        print(f'dtau, no converge chicos, dtau[{i}], {dif}, {dtau[i]}') # If it prints, increase N_tau


# BODY

v1 = np.array([[dtau[j] / dS[i] ** 2 for j in range(N_tau)] for i in range(N_S)])
v2 = np.array([[dtau[j] / dS[i] for j in range(N_tau)] for i in range(N_S)])
'''
# Apply upwind differencing (using forward or backward differencing for Delta depending on the sign of b), not necessarily better
positive_b = b >= 0
A = np.array([[v1[i, j] * a[i, j]     - (~positive_b)[i, j] * v2[i, j] * (b[i, j] + b[i - 1, j]) / 2     for j in range(N_tau)] for i in range(N_S)])
B = np.array([[-2 * v1[i, j] * a[i, j] + dtau[j] * c[i, j]     - positive_b[i, j] * v2[i, j] * (b[i, j] + b[i + 1, j]) / 2     + (~positive_b)[i, j] * v2[i, j] * (b[i, j] + b[i - 1, j]) / 2     for j in range(N_tau)] for i in range(N_S)])
C = np.array([[v1[i, j] * a[i, j]     + positive_b[i, j] * v2[i, j] * (b[i, j] + b[i + 1, j]) / 2     for j in range(N_tau)] for i in range(N_S)])
#'''
#'''
# Normal formulas for A, B, C, using central difference for Delta:
A = np.array([[v1[i, j] * a[i, j] - (1/2) * v2[i, j] * b[i, j] for j in range(N_tau)] for i in range(N_S)])
B = np.array([[-2 * v1[i, j] * a[i, j] + dtau[j] * c[i, j] for j in range(N_tau)] for i in range(N_S)])
C = np.array([[v1[i, j] * a[i, j] + (1/2) * v2[i, j] * b[i, j] for j in range(N_tau)] for i in range(N_S)])
#'''

# Option value matrix:
V_matrix = np.zeros((N_S + 1, N_tau + 1)) # Initialize matrix
# Fill values at maturity
for i in range(N_S + 1):
    V_matrix[i, 0] = Payoff(S_array[i])
# Fill values at boundaries
for i in range(N_tau + 1):
    V_matrix[0, i] = LBoundary(0, i)
    V_matrix[N_S, i] = UBoundary(N_S, i)
# Fill remaining values
for i in range(N_tau):
    for j in range(1, N_S):
        V_matrix[j, i + 1] = A[j, i] * V_matrix[j - 1, i] + (1 + B[j, i]) * V_matrix[j, i] + C[j, i] * V_matrix[j + 1, i]

current_value = V_matrix[N_S // 2, N_tau]


# Error code
'''
from scipy.stats import norm

S_array[0] = 1e-14
def option(op_class, E, T, sigma, r, S, D = 0, t = 0):
    d1 = (math.log(S / E) + (r - D + (1/2) * sigma ** 2) * (T - t)) / (sigma * math.sqrt(T - t))
    d2 = d1 - sigma * math.sqrt(T - t)
    if (op_class == "C"):
        return S * math.exp(-D * (T - t)) * norm.cdf(d1) - E * math.exp(-r * (T - t)) * norm.cdf(d2)
    elif (op_class == "P"):
        return -S * math.exp(-D * (T - t)) * norm.cdf(-d1) + E * math.exp(-r * (T - t)) * norm.cdf(-d2)
    else:
        raise ValueError(f'Please introduce a valid op_class: either "C" (Call) or "P" (Put)')

error_matrix = np.array([[V_matrix[i, j] - option("C", 100, tau_array[j], 0.2, 0.04, S_array[i]) for j in range(N_tau + 1)] for i in range(N_S + 1)])
current_error = error_matrix[N_S // 2, N_tau]
print(current_error)
print(error_matrix)
#'''

np.set_printoptions(suppress=True)   # disables scientific notation
np.set_printoptions(precision=3)
'''
np.set_printoptions(suppress=False)
np.set_printoptions(precision=8)
#'''

print(current_value)
print(V_matrix)






# Bilinear interpolation function

def bilinear_interpolation(S, tau, matrix, tau_axis, S_axis):
    '''
    Uses bilinear interpolation to find V for values of S and tau not solved for in the discrete numerical solution.

    Parameters:
        S (float): Asset value coordinate of the option value you want to find.
        tau (float): Time-to-maturity coordinate of the option value you want to find.
        matrix (2-dimensional np.ndarray): Numerical solution matrix for V, at discrete points.
        tau_axis (list or np.ndarray): Discrete valued tau-axis of the matrix.
        S_axis (list or np.ndarray): Discrete valued S-axis of the matrix.
    
    Returns:
        float: Value of the option at tau and S given, V(S, tau).
    '''
    # Make sure valid bounds
    if (tau < tau_axis[0] or tau > tau_axis[-1]):
        raise ValueError(f'tau out of bounds, please introduce one inside the bounds. Bounds: [{tau_axis[0]}, {tau_axis[-1]}]')
    elif (S < S_axis[0] or S > S_axis[-1]):
        raise ValueError(f'S out of bounds, please introduce one inside the bounds. Bounds: [{S_axis[0]}, {S_axis[-1]}]')
    
    # Cover all cases regarding if the inputs are values inside the coordinate axis
    if np.any(np.isclose(tau_axis, tau)) and np.any(np.isclose(S_axis, S)): # If the point already exists in the matrix (just extract the solution)

        # Find the inidces of the S- and tau-coordinate
        i_S = np.where(np.isclose(S_axis, S))[0][0]
        i_tau = np.where(np.isclose(tau_axis, tau))[0][0]

        return matrix[i_S, i_tau]
    
    elif np.any(np.isclose(tau_axis, tau)): # If only the tau-xoordinate is in the tau-axis (linear approximation)

        # Find the indices of the tau-coordinate
        i_tau = np.where(np.isclose(tau_axis, tau))[0][0]

        # Find the indices of the S values bounding the point
        i_S = 0
        for i in range(1, len(S_axis)):
            if S_axis[i] > S:
                i_S = i - 1
                break
        
        return (matrix[i_S, i_tau] * (S_axis[i_S + 1] - S) + matrix[i_S + 1, i_tau] * (S - S_axis[i_S])) / (S_axis[i_S + 1] - S_axis[i_S])
    
    elif np.any(np.isclose(S_axis, S)): # If only the S-xoordinate is in the S-axis (linear approximation)

        # Find the indices of the S-coordinate
        i_S = np.where(np.isclose(S_axis, S))[0][0]

        # Find the indices of the tau values bounding the point
        i_tau = 0
        for i in range(1, len(tau_axis)):
            if tau_axis[i] > tau:
                i_tau = i - 1
                break
        
        return (matrix[i_S, i_tau] * (tau_axis[i_tau + 1] - tau) + matrix[i_S, i_tau + 1] * (tau - tau_axis[i_tau])) / (tau_axis[i_tau + 1] - tau_axis[i_tau])
    
    else: # If neither coordinate is in the axis (bilinear interpolation)

        # Find the indices of the S and tau values bounding the point
        i_S = 0
        i_tau = 0
        for i in range(1, len(S_axis)):
            if S_axis[i] > S:
                i_S = i - 1
                break
        for i in range(1, len(tau_axis)):
            if tau_axis[i] > tau:
                i_tau = i - 1
                break
        
        # Calculate areas and extract option values in a correct order to apply the bilinear interpolation formula
        Area = np.array([
            (S_axis[i_S + 1] - S) * (tau_axis[i_tau + 1] - tau),
            (S_axis[i_S + 1] - S) * (tau - tau_axis[i_tau]),
            (S - S_axis[i_S]) * (tau_axis[i_tau + 1] - tau),
            (S - S_axis[i_S]) * (tau - tau_axis[i_tau])
            ])
        V = np.array([matrix[i_S, i_tau], matrix[i_S, i_tau + 1], matrix[i_S + 1, i_tau], matrix[i_S + 1, i_tau + 1]])

        return sum(Area * V) / sum(Area)



10.798807243201244
[[  0.      0.      0.    ...   0.      0.      0.   ]
 [  0.      0.      0.    ...   0.      0.      0.   ]
 [  0.      0.      0.    ...   0.      0.      0.   ]
 ...
 [159.483 159.498 159.513 ... 164.536 164.543 164.55 ]
 [164.689 164.705 164.719 ... 169.207 169.214 169.22 ]
 [170.    170.007 170.013 ... 173.908 173.915 173.921]]
[0.    0.002 0.003 0.005 0.007 0.008 0.01  0.012 0.013 0.015 0.017 0.018
 0.02  0.022 0.023 0.025 0.027 0.028 0.03  0.032 0.033 0.035 0.037 0.038
 0.04  0.042 0.043 0.045 0.047 0.048 0.05  0.052 0.053 0.055 0.057 0.058
 0.06  0.062 0.063 0.065 0.067 0.068 0.07  0.072 0.073 0.075 0.077 0.078
 0.08  0.082 0.083 0.085 0.087 0.088 0.09  0.092 0.093 0.095 0.097 0.098
 0.1   0.102 0.103 0.105 0.107 0.108 0.11  0.112 0.113 0.115 0.117 0.118
 0.12  0.122 0.123 0.125 0.127 0.128 0.13  0.132 0.133 0.135 0.137 0.138
 0.14  0.142 0.143 0.145 0.147 0.148 0.15  0.152 0.153 0.155 0.157 0.158
 0.16  0.162 0.163 0.165 0.167 0.168 0.17  0.172 0.173 0.175 

C:\Users\Umbi Pumbi\AppData\Local\Temp\ipykernel_356\2100644280.py:48: RuntimeWarning: invalid value encountered in scalar divide
  dif = min([2 * a[i, j] / abs(b[i, j]) for j in range(N_tau)]) - dS[i]
C:\Users\Umbi Pumbi\AppData\Local\Temp\ipykernel_356\2100644280.py:53: RuntimeWarning: divide by zero encountered in scalar divide
  dif = min([dS[j] / (2 * a[j, i]) for j in range(N_S)]) - dtau[i]


## (2) Efficiently applied to Black-Scholes and other assumptions
The other assumptions are constant risk-free rate and volatility, custom S axis (starts linear, then goes lognormal) and equal time steps.

In [30]:
'''
### (3 hashtags) means that this line can be customized for the particular case, and the code is generalized enough to adapt.
'''

# PARAMETERS
# Financial parameters
S_0 = 100 ### Current asset value
T = 1 ### In years
sigma = 0.2 ### In years^(-1/2)
r = 0.04 ### In years^(-1)

# Option parameters
def Payoff(S): ###
    return max(S - 100, 0)
def LBoundary(S, tau): ###
    return 0
def UBoundary(S, tau): ###
    return S - 100 * math.exp(-r * tau)

# Method parameters
N_S = 100 ### Number of asset steps apart from the 0th
N_tau = 600 ### Number of time steps apart from the 0th

# Axis
k = max(math.exp(r * T) * (1 + 8 * sigma), (1 + 1 / (N_S // 2)))
S_array = np.array([i * (S_0 / (N_S // 2)) for i in range(N_S // 2)] + [S_0 * k ** (2 * i / N_S) for i in range(math.ceil(N_S / 2) + 1)])
dS = np.array([S_array[i + 1] - S_array[i] for i in range(N_S)])

dtau = T / N_tau
tau_array = np.array([dtau * i for i in range(N_tau + 1)]) # Time to maturity (in years), so tau_array[0] = 0 is maturity, and tau_array[N_tau] = T is current time

# Equation parameters
a = np.array([(1/2) * (sigma * S_array[i]) ** 2 for i in range(N_S + 1)])
b = np.array([r * S_array[i] for i in range(N_S + 1)])
c = -r

# Convergence constraints
for i in range(N_S):
    dif = 2 * a[i] / abs(b[i]) - dS[i]
    if dif < 0:
        print(f'dS, no converge chicos, dS[{i}], {dif}, {dS[i]}') # If it prints, increase N_S

for i in range(N_tau):
    dif = min([dS[j] / (2 * a[j]) for j in range(N_S)]) - dtau
    if dif < 0:
        print(f'dtau, no converge chicos, dtau[{i}], {dif}, {dtau}') # If it prints, increase N_tau

# BODY
v1 = dtau / np.array([dS[i] ** 2 for i in range(N_S)])
v2 = dtau / np.array([dS[i] for i in range(N_S)])
A = np.array([v1[i] * a[i] - (1/2) * v2[i] * b[i] for i in range(N_S)])
B = np.array([-2 * v1[i] * a[i] + dtau * c for i in range(N_S)])
C = np.array([v1[i] * a[i] + (1/2) * v2[i] * b[i] for i in range(N_S)])

# Option value matrix:
V_matrix = np.zeros((N_S + 1, N_tau + 1)) # Initialize matrix
# Fill values at maturity
for i in range(N_S + 1):
    V_matrix[i, 0] = Payoff(S_array[i])
# Fill values at boundaries
for i in range(N_tau + 1):
    V_matrix[0, i] = LBoundary(S_array[0], tau_array[i])
    V_matrix[N_S, i] = UBoundary(S_array[N_S], tau_array[i])
# Fill remaining values
for i in range(N_tau):
    for j in range(1, N_S):
        V_matrix[j, i + 1] = A[j] * V_matrix[j - 1, i] + (1 + B[j]) * V_matrix[j, i] + C[j] * V_matrix[j + 1, i]

current_value = V_matrix[N_S // 2, N_tau]


# Error code
'''
from scipy.stats import norm
np.set_printoptions(suppress=True)   # disables scientific notation
np.set_printoptions(precision=3)
# np.set_printoptions()

S_array[0] = 1e-14
def option(op_class, E, T, sigma, r, S, D = 0, t = 0):
    d1 = (math.log(S / E) + (r - D + (1/2) * sigma ** 2) * (T - t)) / (sigma * math.sqrt(T - t))
    d2 = d1 - sigma * math.sqrt(T - t)
    if (op_class == "C"):
        return S * math.exp(-D * (T - t)) * norm.cdf(d1) - E * math.exp(-r * (T - t)) * norm.cdf(d2)
    elif (op_class == "P"):
        return -S * math.exp(-D * (T - t)) * norm.cdf(-d1) + E * math.exp(-r * (T - t)) * norm.cdf(-d2)
    else:
        raise ValueError(f'Please introduce a valid op_class: either "C" (Call) or "P" (Put)')

error_matrix = np.array([[V_matrix[i, j] - option("C", 100, tau_array[j], 0.2, 0.04, S_array[i]) for j in range(N_tau + 1)] for i in range(N_S + 1)])
current_error = error_matrix[N_S // 2, N_tau]
print(current_error)
print(error_matrix)
#'''

print(current_value)
print(V_matrix)

C:\Users\Umbi Pumbi\AppData\Local\Temp\ipykernel_356\1321729385.py:39: RuntimeWarning: invalid value encountered in scalar divide
  dif = 2 * a[i] / abs(b[i]) - dS[i]
C:\Users\Umbi Pumbi\AppData\Local\Temp\ipykernel_356\1321729385.py:44: RuntimeWarning: divide by zero encountered in scalar divide
  dif = min([dS[j] / (2 * a[j]) for j in range(N_S)]) - dtau


10.808374363387731
[[  0.      0.      0.    ...   0.      0.      0.   ]
 [  0.      0.      0.    ...   0.      0.      0.   ]
 [  0.      0.      0.    ...   0.      0.      0.   ]
 ...
 [160.047 160.062 160.077 ... 165.104 165.112 165.119]
 [165.276 165.291 165.306 ... 169.796 169.803 169.81 ]
 [170.611 170.617 170.624 ... 174.519 174.525 174.532]]
